In [1]:
import numpy as np
import pandas as pd
import torch


In [2]:
import os

In [3]:
import pickle

In [4]:
import pylangacq as pla

In [5]:
import random

Load Targets

In [53]:
# y_cookie = torch.load('cookie_data_targets.pt')

In [6]:
y_cookie = torch.load('cookie_target_dict.pt')

In [7]:
len(y_cookie)

552

In [56]:
# y_sentence = torch.load('sentence_data_targets.pt')

In [8]:
y_sentence = torch.load('sentence_target_dict.pt')

In [9]:
len(y_sentence)

240

Load Samples

In [10]:
X_cookie_filenames = torch.load('cookie_fnames_dict.pt')

In [127]:
# Group not listed but control
X_cookie_filenames[219]

'/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha'

In [129]:
y_cookie[219]

''

In [131]:
X_cookie_reader.headers()[X_cookie_filenames[219]]['Participants']['PAR']['group']

''

In [11]:
len(X_cookie_filenames)

552

In [12]:
X_sent_filenames = torch.load('sentence_fnames_dict.pt')

In [13]:
len(X_sent_filenames)

240

In [14]:
num_samples = len(X_cookie_filenames) + len(X_sent_filenames)

In [15]:
num_samples

792

In [16]:
X_cookie_reader = pla.read_chat('./Pitt/*/cookie/*.cha')
X_sentence_reader = pla.read_chat('./Pitt/*/sentence/*.cha')

-----------------------------------

Missing data -- targets

group missing or input error st no mean to replace with and left untouched in target generation

replace with closest analog as per means generated from target generation:

cookie {'Control': 30, 'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 20, 'Probable': 19, 'Other': 24}

sentence {'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 21, 'Control': 30, 'Probable': 19, 'Other': 24}

In [137]:
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])

/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/236-0.cha
674 Dementia


In [142]:
target_674 = X_sentence_retder.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/236-0.cha']['Participants']['PAR']['group']

In [143]:
target_674

''

In [144]:
group_674

'Dementia'

In [154]:
# Replace Dementia with Possible mean (as per diagnostic codes in Pitt-data.xlxs and accompanying Pitt-readme.pdf)
y_sentence[674] = 21

In [139]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha
219 
/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha
511 possibleAD


In [146]:
target_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['education']
group_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['group']

In [147]:
target_219

''

In [148]:
group_219

''

In [156]:
# Replace '' with Control mean (as per folder and dx codes in .xlxs)
y_cookie[219] = 30

In [149]:
target_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['education']
group_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['group']

In [150]:
target_511

''

In [151]:
group_511

'possibleAD'

In [158]:
# Replace possibleAD with PossibleAD mean
y_cookie[511] = 20

In [159]:
# confirm all targets filled now 
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])


In [160]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

In [161]:
# torch.save(y_cookie,'cookie_target_dict.pt')

In [162]:
# torch.save(y_sentence,'sentence_target_dict.pt')

---------------------------------------------------------------

Load Vocab Embeddings

In [17]:
glove_path = './PretrainedWordEmb/'

In [18]:
vocab_emb = pickle.load(open(f'{glove_path}/addb.vocab_emb.glove.42B.300.pkl', 'rb'))

In [19]:
pos_dict = torch.load('pos_dict.pt')

In [209]:
def get_minibatch(batchsize=1, ix=None):
    
    minibatch_ix = random.sample(range(num_samples),batchsize) if ix is None else ix 
    
    #index, filename
    cookie_files = [(i,X_cookie_filenames[i]) for i in minibatch_ix if i in X_cookie_filenames.keys()]
    sent_files = [(i,X_sent_filenames[i] )for i in minibatch_ix if i in X_sent_filenames.keys()]
    
    embeddings = []
    targets = []

    for corpus,data,targetdict in [(cookie_files,X_cookie_reader,y_cookie),(sent_files,X_sentence_reader,y_sentence)]: 
        for file_ix,file in corpus:
#             print(file_ix, file)
#             print('Words',[tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])
#             print('Words_len',len([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance]))
            embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])]
            target = targetdict[file_ix]
            
            for tkn in embedding:

                tkn[1][tkn[2]] = 1

#                 embeddings.append(torch.cat((tkn[0],tkn[1])))
            
            embeddings.append([torch.cat((tkn[0],tkn[1])) for tkn in embedding])


#             embeddings.append(embedding)
            targets.append(target)
            
#     return embeddings, torch.tensor(targets), minibatch_ix
    return embeddings, torch.tensor(targets)

In [64]:
range(num_samples)

range(0, 792)

In [189]:
e,t,i = get_minibatch(batchsize=num_samples)

In [190]:
len(e)

792

In [191]:
len(t)

792

In [197]:
# targets are [1,30]
set([30, 18, 29, 12, 22, 16, 23, 28, 30, 21, 29, 24, 30, 20, 19, 30, 30, 30,
        19, 30, 18, 20, 24, 30, 30, 20, 29, 30, 22, 22, 30, 19, 19, 30, 29, 17,
        30, 23, 19, 18, 19, 24, 24, 26, 27, 16, 23, 29, 22, 19, 27, 28, 29, 20,
        17, 30, 16, 20, 28, 29, 25, 12, 17, 30, 27, 30, 30, 18, 28, 11, 30, 30,
        19, 13, 10, 20, 30, 30, 20, 30, 16, 29, 21, 30, 19, 28, 13, 23, 28, 28,
        19, 25, 23, 30, 20, 29, 30, 28, 28, 26, 29, 19, 30, 30, 29, 29, 25, 23,
        30, 19, 19, 29, 30, 19, 15, 30, 23, 17, 30, 30, 19, 30, 29, 19, 18, 25,
        30, 30, 30, 30, 13, 30, 26, 29, 30, 14, 20, 30, 30, 13, 28, 23, 24, 17,
        14, 13, 27, 30, 30, 29, 30, 13, 25, 30, 15, 18, 24, 25, 29, 16, 29, 20,
        29, 29, 23, 21, 28, 15, 20, 24, 30, 29, 29, 12, 25, 28, 28, 30, 11, 15,
        30, 29, 27, 18, 27, 30, 17, 29, 28, 27, 28, 30, 19, 17, 19, 24, 13, 21,
        19, 20, 23, 30, 30, 19, 13, 19, 28, 28, 13, 29, 29, 30, 29, 22, 26, 23,
        30, 14, 19, 26,  8, 19, 19, 28, 30, 15, 11, 16, 28, 17, 21, 30, 19, 15,
        29, 30, 26, 30, 18, 29, 29, 30, 30, 30, 18, 20, 30, 26, 29, 12, 19, 26,
        25, 30, 30, 30, 19, 26, 15, 19, 17, 30, 30, 16, 30, 30, 30, 11, 24, 29,
        26, 23, 19, 23, 18, 15, 30, 26, 12, 17, 13, 29, 22, 23, 30, 28, 28, 30,
        29, 29, 16, 18, 20, 30, 30, 30, 22, 30, 29, 25, 30, 30, 30, 24, 23, 20,
        10, 29, 30, 24, 15, 27, 27, 28, 30, 27, 27, 22, 19, 30, 17, 19, 18, 30,
        30, 30, 29, 26, 29, 28, 18, 30, 10, 19, 30, 30, 13, 27, 29, 30, 24, 30,
        29, 17, 19,  7, 12, 29, 30, 20, 23, 30, 30, 11, 30, 28, 30, 25, 12, 30,
        30, 10, 20, 30, 30, 26, 15, 30, 19, 29, 30, 18, 30, 17, 28, 30, 30, 28,
        10, 29, 28, 28, 30, 30, 30, 29,  3, 18, 30, 12, 27, 30, 19, 28, 20, 30,
        30, 30, 30, 30, 27, 29, 19, 15, 27, 17, 28, 13, 14, 28, 30, 30, 27, 30,
        26, 29, 30, 30, 30, 30, 27, 30, 25, 29, 16, 28, 30, 29, 17, 18, 15, 30,
        28, 25, 28, 28, 30, 19, 23, 19, 23, 30, 14, 30, 22, 20, 30, 30, 27, 30,
        30, 21, 10, 30, 30, 28, 18, 13, 29, 29, 29, 17, 30, 17, 19, 26, 30, 28,
        30, 19, 27, 30, 30, 20, 27, 10, 21,  5, 30, 17, 17, 24, 20, 29, 23, 28,
        29, 25, 28, 30, 20, 30, 30, 29, 20, 28, 28, 30, 15, 30, 30, 20, 20, 18,
        19, 20, 23, 10, 21, 20, 30, 30, 17,  8, 30, 27, 13, 30, 30, 14, 23, 28,
        30, 30, 22, 28, 24, 25,  3, 30, 19, 29, 30, 28, 27, 20, 29, 25, 20, 30,
        12, 28, 22,  1, 24, 17, 30, 24, 30, 20, 30, 29, 19, 24, 28, 16, 21, 19,
        25, 11, 20, 29, 27, 20, 27, 23, 16, 30, 16, 18, 20, 15, 20,  8, 23, 20,
        19, 20, 20, 10, 18, 23, 16, 28, 15, 12, 21, 19, 28, 28, 27, 22, 18, 18,
        19, 19, 30, 24, 19, 19, 21, 25, 25, 26, 19, 17, 23, 24, 26, 19, 17, 29,
        18, 21, 19, 29, 18, 10, 17, 20, 20, 30, 20, 19, 19, 22, 23, 19, 27, 26,
        21, 19, 10, 18, 25, 19, 17, 23, 28, 19, 26, 20, 18, 20, 23, 29, 19, 16,
        17, 27, 17, 19, 28, 17, 23, 27, 15, 19, 18, 13, 10, 17, 13, 19, 28, 17,
         7, 28, 13, 20, 19, 23, 11, 19, 20, 25, 17,  8, 30, 25, 28, 28, 20, 10,
        17, 19, 24, 17, 24, 15, 22, 29, 26, 13, 12, 12, 27, 27, 23, 25, 24, 25,
        28, 11, 21, 28, 23, 26, 24, 14, 19, 25, 19, 19, 19, 14, 28, 27, 23, 13,
        29, 22, 22, 20, 25, 12, 24, 13, 29, 27, 18, 20, 28, 23, 29, 15, 26, 15,
        24, 18, 13, 11, 17, 11, 19, 20, 17, 26, 20, 18, 23, 17, 15, 22, 12, 13,
        21, 18, 14, 19, 19, 13, 19, 19, 30, 20, 29,  7, 15, 28, 28, 29, 30, 23,
        25, 25, 23, 27, 19, 21, 19, 23, 20, 19, 19, 19, 28, 14, 18, 19, 19, 13])

{1,
 3,
 5,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30}

In [115]:
i_arr = np.array(i)


In [112]:
str_mask = np.array([type(target) == str for target in t])

In [ ]:
np.where()

In [117]:
i_arr[str_mask]

array([788, 504, 695])

In [124]:
for str_target in [788,695]:
    print(X_sent_filenames[str_target])
    print(y_sentence[str_target])

/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/704-0.cha
23
/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/291-2.cha
18


In [265]:
len(e)

792

In [266]:
max([len(emb) for emb in e])

546

In [55]:
t

tensor([29, 24, 29, 30, 17, 26, 30, 20, 23, 30])

In [199]:
# all words same len of emb size 371 
set([len(emb[i]) for emb in e for i in range(len(emb))])

{371}

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Seq2Seq -- with attention

2 layer rnn..

In [200]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [270]:
HIDDEN_SIZE = 128
EMBEDDING_SIZE = 371

In [250]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, emb_size=371, hidden_size=HIDDEN_SIZE):
        super(EncoderRNN, self).__init__()
        
        self.emb_size = emb_size
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, emb_size)
        
        
#         self.gru = nn.GRU(hidden_size, hidden_size)
        self.gru = nn.GRU(input_size=self.emb_size, hidden_size=self.hidden_size,num_layers=2)

    def forward(self, input, hidden):
        # confirm dims here...
#         embedded = self.embedding(input).view(1, 1, -1)
        
        embedded = torch.cat(input).view(len(input), 1, -1)
        
        output = embedded
        
#         output, hidden = self.gru(self.emb_size, self.hidden_size)
        output, hidden = self.gru(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [210]:
x,y = get_minibatch(batchsize=1)

In [215]:
len(x[0])

90

In [217]:
len(y)

1

In [218]:
y

tensor([23])

In [251]:
encoder = EncoderRNN(len(x[0])).to(device)

In [252]:
_hidden = encoder.initHidden()

In [254]:
encoder = encoder.float()

In [313]:
torch.cat([inp.float() for inp in x[0]]).view(len([inp.float() for inp in x[0]]),1,-1).shape

torch.Size([90, 1, 371])

In [258]:
# mod .float() in get_embeddings
# TODO batch right -- or to train on sequences of difference lengths 
out = encoder([inp.float() for inp in x[0]],_hidden)

In [261]:
out[0].shape

torch.Size([90, 1, 128])

In [276]:
out[1].shape

torch.Size([2, 1, 128])

In [274]:
len(out)

2

In [241]:

torch.cat(x[0]).view(len(x[0]), 1, -1).shape

torch.Size([90, 1, 371])

In [245]:

torch.cat(x[0]).view(len(x[0]), 1, -1)[:][1]

torch.Size([1, 371])

In [268]:
# max([len(emb) for emb in e])

546

In [269]:
MAX_LENGTH = 546

In [315]:
class AttnDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
    def __init__(self, hidden_size=HIDDEN_SIZE, output_size=1, dropout_p=0.1, max_length=MAX_LENGTH):    
    
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        
        #fc
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
#         embedded = self.embedding(input).view(1, 1, -1)
#         embedded = torch.cat(input).view(len(input), 1, -1)
        embedded = input
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        # unnecessary...predicting one value
#         output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [316]:
decoder = AttnDecoderRNN()

In [273]:
decoder_hidden = decoder.initHidden()

In [280]:
_hidden.shape

torch.Size([1, 1, 128])

In [287]:
out[0]

tensor([[[ 0.0060, -0.0245, -0.0464,  ...,  0.0695,  0.0342, -0.0551]],

        [[ 0.0537, -0.0173, -0.1303,  ...,  0.0952,  0.0043, -0.0770]],

        [[ 0.0549,  0.0048, -0.1962,  ...,  0.0754, -0.0186, -0.1096]],

        ...,

        [[ 0.1965,  0.0106, -0.3525,  ...,  0.2107, -0.0763, -0.1470]],

        [[ 0.1858, -0.0295, -0.2714,  ...,  0.1975, -0.0800, -0.1494]],

        [[ 0.1433, -0.0282, -0.2725,  ...,  0.1491, -0.0575, -0.1883]]],
       grad_fn=<StackBackward>)

In [290]:
out[0]

tensor([[[ 0.0060, -0.0245, -0.0464,  ...,  0.0695,  0.0342, -0.0551]],

        [[ 0.0537, -0.0173, -0.1303,  ...,  0.0952,  0.0043, -0.0770]],

        [[ 0.0549,  0.0048, -0.1962,  ...,  0.0754, -0.0186, -0.1096]],

        ...,

        [[ 0.1965,  0.0106, -0.3525,  ...,  0.2107, -0.0763, -0.1470]],

        [[ 0.1858, -0.0295, -0.2714,  ...,  0.1975, -0.0800, -0.1494]],

        [[ 0.1433, -0.0282, -0.2725,  ...,  0.1491, -0.0575, -0.1883]]],
       grad_fn=<StackBackward>)

In [293]:
out[0].to(device).long()

tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]])

In [294]:
out[1].to(device).long()

tensor([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [283]:
out[1]

tensor([[[ 1.5723e-01,  1.5999e-01, -1.5316e-01,  1.7675e-01, -2.5812e-01,
           4.7489e-01,  2.1191e-01, -4.6458e-02, -1.0214e-01,  2.5158e-01,
           3.6603e-01,  2.6250e-02, -4.6524e-02, -2.0161e-01, -2.7700e-01,
          -5.7927e-02,  1.9463e-01,  1.3949e-01, -2.8001e-01, -2.0826e-01,
           6.8004e-01, -1.4004e-01,  1.0250e-01,  2.1201e-01,  3.5271e-01,
          -1.4978e-01, -3.3812e-01, -3.2099e-02, -2.5270e-01, -3.1943e-01,
           7.4798e-02,  6.0758e-01, -5.9771e-01, -1.0965e-01,  1.8242e-01,
           1.0807e-01,  1.1915e-01,  5.6361e-01,  3.9456e-01,  7.9710e-03,
          -4.7770e-01,  1.7498e-01, -5.0252e-01,  4.2966e-02,  1.0658e-01,
          -9.2730e-02,  9.1547e-02,  7.7902e-03, -3.4490e-01,  3.5609e-01,
           2.9160e-01,  3.4331e-01, -9.6344e-02,  3.8138e-01, -2.2851e-01,
          -2.5756e-01,  3.5514e-01,  2.0115e-01,  1.1860e-01,  1.0311e-02,
           2.0202e-01, -4.0456e-01,  6.0682e-02, -2.0456e-01, -2.1061e-02,
          -3.1327e-01,  2

In [301]:
out[1]

tensor([[[ 1.5723e-01,  1.5999e-01, -1.5316e-01,  1.7675e-01, -2.5812e-01,
           4.7489e-01,  2.1191e-01, -4.6458e-02, -1.0214e-01,  2.5158e-01,
           3.6603e-01,  2.6250e-02, -4.6524e-02, -2.0161e-01, -2.7700e-01,
          -5.7927e-02,  1.9463e-01,  1.3949e-01, -2.8001e-01, -2.0826e-01,
           6.8004e-01, -1.4004e-01,  1.0250e-01,  2.1201e-01,  3.5271e-01,
          -1.4978e-01, -3.3812e-01, -3.2099e-02, -2.5270e-01, -3.1943e-01,
           7.4798e-02,  6.0758e-01, -5.9771e-01, -1.0965e-01,  1.8242e-01,
           1.0807e-01,  1.1915e-01,  5.6361e-01,  3.9456e-01,  7.9710e-03,
          -4.7770e-01,  1.7498e-01, -5.0252e-01,  4.2966e-02,  1.0658e-01,
          -9.2730e-02,  9.1547e-02,  7.7902e-03, -3.4490e-01,  3.5609e-01,
           2.9160e-01,  3.4331e-01, -9.6344e-02,  3.8138e-01, -2.2851e-01,
          -2.5756e-01,  3.5514e-01,  2.0115e-01,  1.1860e-01,  1.0311e-02,
           2.0202e-01, -4.0456e-01,  6.0682e-02, -2.0456e-01, -2.1061e-02,
          -3.1327e-01,  2

In [305]:
out1tup = (torch.tensor(out[1][0]),torch.tensor(out[1][1]))

/Users/renee/opt/anaconda3/envs/dl1010/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [306]:
out1tup[0]

tensor([[ 0.1572,  0.1600, -0.1532,  0.1767, -0.2581,  0.4749,  0.2119, -0.0465,
         -0.1021,  0.2516,  0.3660,  0.0262, -0.0465, -0.2016, -0.2770, -0.0579,
          0.1946,  0.1395, -0.2800, -0.2083,  0.6800, -0.1400,  0.1025,  0.2120,
          0.3527, -0.1498, -0.3381, -0.0321, -0.2527, -0.3194,  0.0748,  0.6076,
         -0.5977, -0.1096,  0.1824,  0.1081,  0.1191,  0.5636,  0.3946,  0.0080,
         -0.4777,  0.1750, -0.5025,  0.0430,  0.1066, -0.0927,  0.0915,  0.0078,
         -0.3449,  0.3561,  0.2916,  0.3433, -0.0963,  0.3814, -0.2285, -0.2576,
          0.3551,  0.2011,  0.1186,  0.0103,  0.2020, -0.4046,  0.0607, -0.2046,
         -0.0211, -0.3133,  0.2168, -0.3272, -0.1296, -0.0896,  0.1216, -0.1395,
          0.1782, -0.1907, -0.3478, -0.4013, -0.3843, -0.3145, -0.3294,  0.0402,
          0.1601, -0.0487,  0.0034,  0.2241, -0.1563,  0.0397, -0.3412,  0.0534,
         -0.2112,  0.1979, -0.0010,  0.0664, -0.1111,  0.0013, -0.4389,  0.3041,
          0.0801,  0.4709, -

In [308]:
out1tup[1]

tensor([[ 1.4330e-01, -2.8160e-02, -2.7253e-01, -2.1159e-01, -9.1865e-02,
          2.3016e-01, -1.9326e-01,  5.3869e-02,  6.9442e-02,  2.9996e-02,
         -1.5446e-01, -2.0936e-02,  1.6643e-01,  1.5336e-02,  1.1667e-03,
          1.7989e-01, -1.8992e-02,  2.1255e-01, -3.1563e-04, -1.4813e-02,
         -2.8875e-02, -7.8578e-02, -3.3935e-02, -1.3566e-01,  7.6635e-02,
          1.6937e-01, -2.1261e-01,  1.6421e-02, -8.6880e-02,  1.6153e-01,
         -1.4032e-01, -2.8072e-01,  1.2293e-01,  8.0072e-02, -6.6193e-02,
         -1.7853e-01, -2.9315e-03, -2.2318e-01, -3.0703e-01, -2.2590e-01,
         -4.3205e-03, -2.9151e-01, -8.4244e-02, -1.0588e-01,  1.1057e-01,
          2.0269e-01,  9.5587e-02, -2.4270e-01,  1.1075e-03, -3.3889e-01,
          2.7359e-01, -1.1423e-01, -3.2622e-02, -3.8472e-03,  1.1763e-01,
          1.4291e-01,  7.4120e-02, -1.3591e-01, -5.0060e-02, -1.3046e-01,
          1.0717e-01, -2.6232e-01,  1.8451e-01,  1.7628e-01,  3.3406e-02,
         -7.5508e-02,  2.8791e-01, -5.

In [317]:
decoder(out[0],out[1],out[1])

RuntimeError: Expected 3-dimensional tensor, but got 4-dimensional tensor for argument #2 'batch2' (while checking arguments for bmm)

In [311]:
out[0].shape

torch.Size([90, 1, 128])

In [312]:
torch.cat(out[0])

TypeError: cat(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [318]:
out[1].shape

torch.Size([2, 1, 128])